In [7]:
import random
import numpy as np

In [24]:
import numpy as np
import pandas as pd
import random 
import matplotlib.pyplot as plt

# nb max de clients est 130
n_clients=50
def data_prep(n_clients):
    n_depot = n_clients+1
    #on importe les donnée sur les clients et le depot
    df=pd.read_excel("2_detail_table_customers.xls")
    df_depot=pd.read_excel('4_detail_table_depots.xls')
    #on extrait les informations qui nous intéressent
    Columns=["CUSTOMER_NUMBER","CUSTOMER_LATITUDE","CUSTOMER_LONGITUDE"]
    Data =( df[Columns]
           .rename(columns={"CUSTOMER_NUMBER":"number","CUSTOMER_LATITUDE":"x","CUSTOMER_LONGITUDE":"y"})
          )
    Data=Data.head(n_clients)
    depot = df_depot[['DEPOT_CODE','DEPOT_LATITUDE','DEPOT_LONGITUDE']].drop_duplicates()
    depot = depot.rename(columns={"DEPOT_CODE":"number of client/depot","DEPOT_LATITUDE":"x","DEPOT_LONGITUDE":"y"})
    Data.loc[len(Data)]=depot.iloc[0]
    return Data
def client_distances(Data):
    #définition d'une matrice U qui contient les distances entre les clienst/depot
    U=np.zeros((len(Data),len(Data)))
    for i in range(len(Data)):
        for j in range(i+1,len(Data)):
            U[i,j]=np.sqrt((Data.iloc[i,1]-Data.iloc[j,1])**2+(Data.iloc[i,2]-Data.iloc[j,2])**2)
            U[j,i]=U[i,j]
    return U
Data=data_prep(n_clients)
U=client_distances(Data)


def fonction_objectif( L,M=U):
    if L==None:
        return 100
    time=0
    V=30
    for i in range(len(L)-1):
        time+=U[L[i]-1,L[i+1]-1]
    return time

In [5]:
def epsilon_greedy_policy(nb_states, state, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        # Exploration : choisir une action aléatoire
        return np.random.randint(nb_states)
    else:
        # Exploitation : choisir l'action avec la valeur Q maximale
        return np.argmax(Q[state])

In [22]:
nb_states=8
nb_actions=8
epsilon=0.5
alpha=0.1 #learning rate
gamma=0.9 #discount rate
num_episodes=20

def Q_learning(num_episodes, alpha, gamma, epsilon, reward):
    Q = np.zeros(nb_states)  # Initialiser la table Q
    for episode in range(num_episodes):
        state = np.randint(1,9)  # État initial
        while not terminal_state(state):
            action = epsilon_greedy_policy(nb_actions, state, epsilon)  # Choix de l'action
            sol, next_state, next_action, reward = apply_action(state, action)  # Application de l'action
            Q[state,action]=(1-alpha)*Q[state,action]+alpha*(reward+gamma*Q[next_state, next_action])
            state=next_state
    return Q 

In [27]:
#pour passer d'un état à un autre avoir - la fonction objectif dans Q

def apply_action(sol, action):
    # Appliquer l'action et calculer la récompense
    if action == 1:
        sol = intra_route_swap(sol)
    elif action == 2:
        sol = inter_route_swap(sol)
    elif action == 3:
        sol = intra_route_shift(sol)
    elif action == 4:
        sol = inter_route_shift(sol)
    elif action == 5:
        sol = two_intra_route_swap(sol)
    elif action == 6:
        sol = two_intra_route_shift(sol)
    elif action == 7:
        sol = eliminate_smallest_route(sol)
    elif action == 8:
        sol = eliminate_random_route(sol)
    
    next_state=action
    next_action = epsilon_greedy_policy(nb_states, next_state, epsilon)
    # Calculer la récompense inversée
    reward_inverse = fonction_objectif(sol)
    
    # Retourner la nouvelle solution et la récompense inversée
    return sol, next_state, next_action, -reward_inverse


In [9]:
#fonctions pour les 8 actions
#action changer d'une fonction de voisinage à un autre
nb_véhicules=2

# 1. Intra-Route Swap: échange d'un client avec un autre client dans la même route
def intra_route_swap(sol):
    # Sélectionner une route au hasard
    k=np.random.randint(1,nb_véhicules)
    #Sélectionner 2 clients au hasard
    i, j = random.sample(sol[k], 2)
    # Échanger les positions des clients
    sol[k][i], sol[k][j] = sol[k][j], sol[k][i]
    return sol

In [18]:
# 2. Inter-Route Swap: déplacement d'échange d'un client d’une route avec un client d’une autre route
def inter_route_swap(sol):
    k1, k2 = random.sample(sol, 2)
    # Sélectionner un client de chaque route au hasard
    i1=np.random.randint(1, len(sol[k1]) - 1)
    i2=np.random.randint(1, len(sol[k2]) - 1)
    # Échanger les clients entre les routes
    sol[k1][i1], sol[k2][i2] = sol[k2][i2], sol[k1][i1]
    return sol

In [19]:
# 3. Intra-Route Shift: déplacement d'un client vers une autre position sur la même route
def intra_route_shift(sol):
    # Sélectionner une route au hasard
    k=np.random.randint(1,nb_véhicules)
    i1=np.random.randint(1,len(sol[k1])-1)
    # Déplacer le client vers une autre position dans la route
    client = sol[k].pop(i1)
    j = random.randint(1, len(sol[k1])-1)  # Sélectionner une nouvelle position au hasard
    sol.insert(j, client)
    return sol

In [20]:
# 4. Inter-Route Shift: déplacement d'un client d’une route à une autre
def inter_route_shift(sol):
    k1, k2 = random.sample(sol, 2)
    # Sélectionner un client de la première route et l'insérer dans la deuxième route
    i1=np.random.randint(1,len(sol[k1])-1)
    i2=np.random.randint(1,len(sol[k2])-1)
    client = sol[k1].pop(i1)
    sol[k2].insert(i2, client)
    return sol

In [13]:
# 5. Two Intra-Route Swap: échange de clients sur la même route (deux clients consécutifs sont échangés avec deux autres clients consécutifs de la même route)
def two_intra_route_swap(sol):
    k=np.random.randint(1,nb_véhicules)
    # Sélectionner deux paires de clients consécutifs au hasard dans la route
    i, j, u, l = random.sample(sol[k], 4)
    # Échanger les paires de clients consécutifs
    sol[k][i], sol[k][j], sol[k][u], sol[k][l] = sol[k][u], sol[k][l], sol[k][i], sol[k][j]
    return sol

In [21]:
# 6. Two Intra-Route Shift: relocalisation des clients sur la même route (deux clients consécutifs sont retirés de leur position et réinsérés dans une autre position de la même route)
def two_intra_route_shift(sol):
    k=np.random.randint(1,nb_véhicules)
    # Sélectionner deux clients consécutifs au hasard dans la route
    i = random.randint(1, len(sol[k])-2)
    # Retirer les deux clients de leur position
    clients = sol[k].pop(i), sol[k].pop(i)
    # Insérer les clients dans une nouvelle position
    j1, j2 = random.sample(sol[k], 2)
    sol[k].insert(j1, clients[0])
    sol[k].insert(j2, clients[1])
    return sol

In [15]:
# 7. Élimine la plus petite route: élimine la plus petite route de la solution
def eliminate_smallest_route(sol):
    # Trouver la plus petite route dans la solution
    smallest_route = min(sol, key=len)
    # Supprimer la plus petite route de la solution
    sol.remove(smallest_route)
    return sol

In [16]:
# 8. Élimine une route aléatoire: élimine une route aléatoire de la solution
def eliminate_random_route(sol):
    # Sélectionner une route au hasard dans la solution
    route_to_remove = random.choice(sol)
    # Supprimer la route de la solution
    sol.remove(route_to_remove)
    return sol

In [25]:
def ChooseAnAction(state, type_function):
    next_state = None
    if type_function == 1:
        next_state = epsilon_greedy_policy(nb_state, state, epsilon)
    elif type_function == 2:
        next_state = np.random.randint(nb_states)
    
    return next_state

In [3]:
def adaptive_local_search_q_learning(x0):
    Q = np.zeros((nb_states, nb_actions))  # Initialiser la table Q
    for episode in range(num_episodes):
        state = 0  # État initial
        improved=True
        no_improvement=0
        x1=x0
        x=x0
        for episode in range(num_episodes):
            reward=0
            states_visited_count=0
            next_state=ChooseAnAction(0,2)
            x=bestNeighbor(next_state,x)
            
        while not terminal_state(state):
            action = epsilon_greedy_policy(Q, state, epsilon)  # Choix de l'action
            next_state, reward = apply_action(state, action)  # Application de l'action
            best_next_action = np.argmax(Q[next_state])  # Meilleure action dans le prochain état
            td_target = reward + discount_factor * Q[next_state, best_next_action]  # Cible TD
            td_error = td_target - Q[state, action]  # Erreur TD
            Q[state, action] += learning_rate * td_error  # Mise à jour de Q
            # Exploration locale
            for _ in range(10):  # Effectuer 10 itérations de recherche locale (modifiable)
                # Effectuer une recherche locale ici (par exemple, permutation d'actions, etc.)
                # Mettre à jour Q en fonction des résultats de la recherche locale
            state = next_state  # Mise à jour de l'état
    return Q


IndentationError: expected an indented block after 'for' statement on line 13 (2558729688.py, line 16)

In [ ]:
def bestNeighbor(next_state,x):
    for i in range()

In [8]:
            best_next_action = np.argmax(Q[next_state])  # Meilleure action dans le prochain état
            td_target = reward + discount_factor * Q[next_state, best_next_action]  # Cible TD
            td_error = td_target - Q[state, action]  # Erreur TD
            Q[state, action] += learning_rate * td_error  # Mise à jour de Q
            state = next_state  # Mise à jour de l'état

TypeError: random_sample() takes at most 1 positional argument (2 given)